In [ ]:
from gurobipy import *
from operator import le
from unicodedata import name
from numpy import conjugate
import random
from gurobipy import *
import pandas as pd

In [ ]:
m1 = Model('VRPTW')

In [ ]:


# m1 = Model('vrptw')

# # 20 customer -> 1.1 minute
# # 30 customers -> 5 minute     1.69 minutes
# # 40                           2.45 min
# # 50 customer -> 160 minute
def mycallback(model, where):

    if where == GRB.Callback.BARRIER:
        # Barrier callback
        itcnt = model.cbGet(GRB.Callback.BARRIER_ITRCNT)
        primobj = model.cbGet(GRB.Callback.BARRIER_PRIMOBJ)
        dualobj = model.cbGet(GRB.Callback.BARRIER_DUALOBJ)
        priminf = model.cbGet(GRB.Callback.BARRIER_PRIMINF)
        dualinf = model.cbGet(GRB.Callback.BARRIER_DUALINF)
        cmpl = model.cbGet(GRB.Callback.BARRIER_COMPL)
        if abs(primobj - dualobj) <= 0.1:
            
             model.terminate()
#         print("%d %g %g %g %g %g" % (itcnt, primobj, dualobj, priminf, dualinf, cmpl))
        

def discreteTime(timeWindow, intervalSize):   # O(lenghtOftimeWindow/size)
    tw = []
    currtime = timeWindow[0]
    while (currtime <= timeWindow[1]):
        tw.append(currtime)
        currtime = currtime+intervalSize
    return tw

In [ ]:
customers = 100
file = "R106.csv"
df = pd.read_csv(file)
start = 0
speed = 1 # km/min
df['XCOORD.'] = df['XCOORD.'] - df['XCOORD.'][0]
df['YCOORD.'] = df['YCOORD.'] - df['YCOORD.'][0]
# df = df[ df['DUE DATE'] >= start + (df['XCOORD.']+df['YCOORD.'])/speed ]
df.reset_index(inplace = True)

serviceTime = df['SERVICE TIME'][1]
#remove unreachable if(starting time + travelling > end )


print(len(df))
# df = df[ df['DUE DATE'] >= start + (df['XCOORD.']+df['YCOORD.'])/speed ]
timeWindows = [] 
for i in range(0,customers):
    st = df['READY TIME'][i+1]
    if(st%10 != 0):
        st=st + (10 - df['READY TIME'][i+1]%10)
    et = df['DUE DATE'][i+1]
    if(et%10 != 0):
        et = et +(10 - df['DUE DATE'][i+1]%10)
    tw = [st,et]
    timeWindows.append(tw)

discreteTimeWindows = []
for i in timeWindows:
    discreteTimeWindows.append(discreteTime(i,10))

    

time_from_source = []
for i in range(0,customers):
    time_from_source.append(round( math.sqrt((df['XCOORD.'][i+1])**2  + (df['YCOORD.'][i+1])**2 ),2))

timeMatrix = []

for i in range(0,customers):
    timeMatrix.append([0]*customers)
for i in range(0,customers):
    for j in range(0,customers):
        if(i==j):
            timeMatrix[i][j] = 0
        elif (i<j):
#             c= int(math.sqrt(abs(df['XCOORD.'][i+1]-df['XCOORD.'][j+1]))+abs(df['YCOORD.'][i+1]-df['YCOORD.'][j+1]))
            euclidean_distance= round( math.sqrt((df['XCOORD.'][i+1]-df['XCOORD.'][j+1])**2  + (df['YCOORD.'][i+1]-df['YCOORD.'][j+1])**2 ),2)
            timeMatrix[i][j] = round(euclidean_distance,2)
            timeMatrix[j][i] = round(euclidean_distance,2)
#             if(c%10 == 0 or c%10 == 5):
#                 timeMatrix[i][j] = c
#                 timeMatrix[j][i] = timeMatrix[i][j]
#             elif(c%10 < 5):
#                 timeMatrix[i][j] = c-(c%10)
#                 timeMatrix[j][i] = timeMatrix[i][j]
#             elif(c%10 > 5):
#                 timeMatrix[i][j] = c+(10 - c%10)
#                 timeMatrix[j][i] = timeMatrix[i][j]      
                
# print("timeWindows",timeWindows)
# print("timeMatrix",timeMatrix)

In [ ]:
# timeWindows = [[30, 80], [10, 40], [50, 90], [80, 110], [60, 80], [100, 110], [30, 40], [40, 100], [60, 120], [40, 50], [10, 50], [100, 160], [90, 110], [80, 90], [40, 80], [80, 110], [10, 30], [30, 40], [10, 70], [10, 20], [70, 80], [80, 90], [70, 130], [10, 30], [20, 70], [80, 140], [30, 90], [80, 130], [20, 70], [90, 150], [70, 80], [50, 70], [70, 100], [80, 100], [70, 120], [90, 140], [80, 90], [80, 110], [90, 140], [20, 80], [10, 70], [80, 120], [30, 80], [10, 60], [30, 80], [50, 70], [20, 70], [90, 100], [60, 70], [60, 80], [80, 130], [60, 100], [70, 80], [90, 120], [70, 80], [100, 140], [100, 160], [10, 30], [50, 90], [60, 120], [100, 140], [60, 100], [40, 50], [100, 150], [100, 150], [80, 90], [90, 140], [100, 120], [70, 130], [10, 60]]

# timeMatrix =  [[0, 60, 30, 20, 70, 20, 90, 100, 10, 70, 40, 50, 40, 70, 40, 70, 70, 40, 80, 30, 40, 30, 50, 10, 40, 10, 60, 40, 100, 10, 60, 90, 60, 20, 10, 10, 40, 70, 20, 10, 20, 70, 80, 50, 20, 30, 40, 90, 100, 10, 40, 50, 30, 60, 70, 80, 70, 10, 60, 40, 90, 70, 90, 80, 70, 70, 30, 100, 50, 30], [60, 0, 50, 50, 60, 100, 30, 70, 80, 20, 90, 70, 70, 20, 40, 30, 100, 90, 30, 40, 10, 100, 10, 10, 100, 90, 100, 50, 60, 90, 10, 40, 10, 80, 50, 80, 50, 20, 80, 40, 10, 10, 40, 10, 20, 50, 100, 20, 30, 50, 30, 100, 60, 30, 40, 80, 40, 10, 10, 70, 80, 80, 90, 30, 20, 80, 50, 100, 60, 50], [30, 50, 0, 10, 90, 30, 40, 70, 50, 70, 90, 40, 60, 80, 50, 70, 50, 80, 10, 80, 100, 40, 10, 40, 100, 10, 80, 10, 80, 90, 100, 50, 60, 30, 60, 70, 90, 20, 70, 10, 40, 20, 100, 30, 90, 40, 70, 10, 60, 10, 30, 60, 90, 10, 70, 80, 40, 60, 60, 70, 90, 30, 40, 60, 100, 40, 70, 30, 10, 30], [20, 50, 10, 0, 100, 30, 30, 70, 20, 40, 70, 100, 80, 90, 90, 100, 100, 70, 70, 90, 70, 60, 60, 90, 70, 20, 80, 80, 60, 30, 20, 60, 20, 100, 100, 50, 10, 30, 100, 80, 100, 100, 90, 100, 20, 100, 10, 50, 60, 60, 80, 100, 80, 60, 60, 40, 70, 40, 60, 40, 30, 80, 30, 90, 50, 20, 50, 90, 50, 60], [70, 60, 90, 100, 0, 70, 10, 90, 70, 80, 60, 100, 40, 100, 40, 70, 30, 90, 40, 40, 80, 10, 90, 70, 30, 60, 30, 90, 70, 40, 40, 20, 60, 20, 10, 70, 10, 90, 60, 90, 80, 10, 90, 10, 100, 20, 30, 30, 60, 40, 100, 50, 40, 10, 80, 40, 80, 50, 30, 60, 10, 50, 10, 80, 70, 10, 40, 10, 50, 30], [20, 100, 30, 30, 70, 0, 60, 100, 30, 80, 70, 80, 90, 90, 50, 70, 80, 30, 100, 10, 20, 40, 30, 100, 90, 30, 80, 50, 50, 80, 30, 40, 90, 80, 50, 60, 30, 50, 50, 90, 10, 20, 30, 40, 100, 50, 40, 60, 10, 20, 10, 20, 30, 80, 20, 70, 40, 90, 10, 10, 30, 70, 70, 50, 50, 10, 70, 50, 60, 20], [90, 30, 40, 30, 10, 60, 0, 50, 50, 50, 40, 20, 30, 10, 20, 30, 60, 70, 80, 100, 60, 20, 60, 20, 10, 50, 90, 20, 20, 20, 20, 20, 100, 60, 60, 70, 50, 50, 80, 100, 60, 10, 100, 30, 70, 40, 80, 60, 60, 90, 70, 70, 80, 50, 100, 60, 90, 10, 40, 90, 40, 40, 50, 100, 100, 40, 40, 20, 10, 20], [100, 70, 70, 70, 90, 100, 50, 0, 50, 40, 90, 40, 60, 10, 10, 20, 50, 50, 90, 20, 100, 80, 40, 40, 50, 100, 50, 60, 40, 60, 40, 10, 80, 50, 90, 50, 30, 90, 100, 20, 100, 20, 90, 20, 30, 50, 10, 80, 20, 90, 10, 60, 30, 20, 70, 100, 40, 70, 10, 30, 20, 30, 100, 30, 80, 60, 40, 50, 80, 70], [10, 80, 50, 20, 70, 30, 50, 50, 0, 20, 70, 40, 40, 70, 80, 100, 90, 40, 20, 70, 30, 80, 100, 20, 90, 30, 40, 90, 70, 100, 10, 10, 60, 40, 80, 90, 40, 10, 40, 80, 10, 100, 50, 100, 80, 70, 100, 30, 60, 50, 10, 60, 30, 90, 60, 10, 70, 50, 90, 30, 50, 70, 100, 20, 60, 70, 20, 70, 90, 100], [70, 20, 70, 40, 80, 80, 50, 40, 20, 0, 60, 20, 90, 60, 100, 60, 80, 30, 20, 90, 70, 100, 70, 60, 30, 50, 80, 70, 60, 70, 70, 80, 70, 20, 50, 40, 30, 80, 50, 100, 90, 100, 20, 10, 50, 80, 60, 70, 30, 60, 70, 70, 50, 90, 40, 90, 60, 50, 90, 30, 90, 20, 50, 10, 80, 50, 80, 90, 60, 80], [40, 90, 90, 70, 60, 70, 40, 90, 70, 60, 0, 70, 80, 90, 100, 50, 10, 90, 40, 70, 80, 60, 60, 50, 100, 20, 90, 30, 100, 30, 50, 100, 40, 10, 90, 40, 60, 30, 60, 30, 90, 50, 40, 10, 20, 70, 30, 80, 50, 40, 100, 50, 40, 50, 90, 30, 70, 90, 80, 70, 60, 100, 70, 20, 20, 50, 20, 100, 80, 40], [50, 70, 40, 100, 100, 80, 20, 40, 40, 20, 70, 0, 50, 60, 50, 100, 40, 10, 80, 50, 100, 40, 30, 20, 20, 40, 80, 90, 70, 50, 100, 20, 60, 100, 40, 10, 60, 90, 10, 90, 80, 10, 40, 80, 30, 10, 20, 80, 50, 20, 20, 70, 100, 20, 70, 70, 50, 40, 30, 80, 90, 40, 90, 100, 100, 10, 60, 70, 70, 80], [40, 70, 60, 80, 40, 90, 30, 60, 40, 90, 80, 50, 0, 40, 10, 70, 20, 70, 20, 100, 10, 50, 90, 40, 70, 50, 50, 70, 100, 80, 40, 50, 100, 50, 100, 40, 70, 100, 60, 20, 90, 50, 20, 20, 40, 70, 60, 10, 70, 60, 30, 70, 20, 90, 50, 70, 90, 70, 60, 100, 30, 50, 90, 60, 20, 40, 70, 100, 20, 80], [70, 20, 80, 90, 100, 90, 10, 10, 70, 60, 90, 60, 40, 0, 90, 60, 100, 20, 50, 40, 20, 10, 80, 50, 20, 40, 80, 50, 60, 70, 60, 80, 50, 100, 70, 60, 50, 20, 40, 30, 50, 10, 100, 50, 80, 80, 40, 50, 40, 40, 70, 10, 50, 100, 80, 100, 90, 30, 60, 80, 10, 40, 50, 70, 80, 40, 60, 100, 60, 100], [40, 40, 50, 90, 40, 50, 20, 10, 80, 100, 100, 50, 10, 90, 0, 40, 60, 30, 20, 20, 20, 10, 50, 100, 20, 40, 30, 70, 40, 60, 70, 10, 10, 90, 50, 80, 10, 10, 30, 30, 40, 10, 10, 80, 20, 60, 60, 50, 30, 100, 20, 30, 80, 50, 100, 50, 90, 30, 70, 30, 70, 90, 80, 10, 80, 40, 10, 50, 50, 30], [70, 30, 70, 100, 70, 70, 30, 20, 100, 60, 50, 100, 70, 60, 40, 0, 40, 90, 100, 90, 70, 90, 80, 70, 90, 70, 70, 20, 80, 40, 60, 30, 100, 20, 10, 20, 60, 90, 10, 10, 10, 70, 70, 50, 70, 100, 50, 60, 60, 60, 40, 20, 20, 80, 40, 40, 60, 70, 40, 20, 70, 70, 40, 20, 100, 90, 70, 10, 30, 30], [70, 100, 50, 100, 30, 80, 60, 50, 90, 80, 10, 40, 20, 100, 60, 40, 0, 80, 20, 20, 50, 50, 60, 20, 40, 40, 70, 30, 60, 70, 20, 80, 20, 30, 30, 20, 30, 10, 60, 40, 50, 70, 20, 40, 70, 40, 10, 10, 20, 60, 90, 50, 90, 30, 40, 100, 10, 60, 100, 30, 70, 60, 20, 70, 50, 40, 50, 40, 60, 90], [40, 90, 80, 70, 90, 30, 70, 50, 40, 30, 90, 10, 70, 20, 30, 90, 80, 0, 90, 50, 30, 30, 80, 20, 30, 70, 70, 100, 20, 80, 20, 60, 50, 20, 40, 70, 50, 30, 70, 20, 30, 30, 70, 10, 60, 80, 100, 40, 50, 90, 100, 70, 30, 100, 90, 90, 40, 80, 20, 10, 50, 40, 40, 90, 10, 60, 40, 50, 90, 80], [80, 30, 10, 70, 40, 100, 80, 90, 20, 20, 40, 80, 20, 50, 20, 100, 20, 90, 0, 60, 40, 90, 40, 40, 80, 80, 80, 90, 60, 100, 70, 60, 10, 100, 70, 70, 100, 30, 30, 40, 10, 90, 50, 20, 30, 90, 60, 90, 40, 90, 100, 70, 100, 10, 10, 10, 90, 50, 20, 60, 70, 10, 60, 20, 20, 80, 90, 80, 60, 10], [30, 40, 80, 90, 40, 10, 100, 20, 70, 90, 70, 50, 100, 40, 20, 90, 20, 50, 60, 0, 30, 10, 70, 10, 30, 80, 70, 100, 100, 90, 40, 80, 50, 90, 70, 90, 40, 30, 90, 70, 10, 40, 30, 30, 70, 70, 10, 40, 20, 30, 80, 60, 70, 20, 10, 20, 80, 80, 80, 100, 100, 30, 100, 30, 20, 70, 70, 30, 70, 10], [40, 10, 100, 70, 80, 20, 60, 100, 30, 70, 80, 100, 10, 20, 20, 70, 50, 30, 40, 30, 0, 40, 40, 100, 70, 60, 90, 50, 30, 60, 10, 40, 20, 30, 40, 90, 90, 10, 70, 30, 90, 80, 60, 50, 30, 50, 40, 90, 50, 60, 10, 80, 100, 80, 90, 50, 70, 100, 60, 70, 50, 70, 90, 30, 80, 60, 90, 20, 100, 80], [30, 100, 40, 60, 10, 40, 20, 80, 80, 100, 60, 40, 50, 10, 10, 90, 50, 30, 90, 10, 40, 0, 10, 40, 50, 10, 20, 70, 10, 80, 100, 100, 60, 50, 70, 50, 60, 60, 30, 90, 50, 100, 30, 20, 100, 70, 20, 50, 10, 70, 40, 70, 40, 30, 70, 70, 10, 60, 60, 20, 40, 80, 90, 90, 10, 70, 20, 30, 70, 60], [50, 10, 10, 60, 90, 30, 60, 40, 100, 70, 60, 30, 90, 80, 50, 80, 60, 80, 40, 70, 40, 10, 0, 30, 10, 40, 10, 20, 60, 10, 90, 40, 90, 10, 60, 80, 80, 40, 40, 90, 20, 10, 70, 20, 60, 80, 100, 40, 80, 70, 20, 20, 80, 60, 60, 100, 80, 60, 80, 90, 100, 40, 90, 80, 100, 50, 40, 70, 70, 40], [10, 10, 40, 90, 70, 100, 20, 40, 20, 60, 50, 20, 40, 50, 100, 70, 20, 20, 40, 10, 100, 40, 30, 0, 70, 10, 30, 70, 80, 20, 90, 80, 10, 70, 40, 70, 90, 70, 80, 20, 20, 20, 90, 10, 30, 70, 30, 30, 100, 80, 60, 100, 10, 90, 80, 20, 20, 70, 100, 100, 30, 10, 70, 80, 70, 50, 30, 80, 40, 30], [40, 100, 100, 70, 30, 90, 10, 50, 90, 30, 100, 20, 70, 20, 20, 90, 40, 30, 80, 30, 70, 50, 10, 70, 0, 70, 40, 40, 10, 50, 40, 40, 60, 50, 80, 10, 30, 90, 90, 70, 60, 40, 40, 100, 20, 70, 10, 50, 100, 20, 10, 80, 20, 100, 90, 60, 20, 80, 100, 50, 60, 100, 80, 30, 80, 100, 80, 20, 60, 20], [10, 90, 10, 20, 60, 30, 50, 100, 30, 50, 20, 40, 50, 40, 40, 70, 40, 70, 80, 80, 60, 10, 40, 10, 70, 0, 20, 100, 10, 80, 30, 20, 80, 10, 10, 40, 40, 100, 30, 100, 20, 60, 90, 20, 60, 50, 60, 70, 90, 20, 50, 20, 50, 10, 70, 90, 40, 50, 40, 100, 30, 60, 60, 60, 80, 100, 10, 50, 10, 10], [60, 100, 80, 80, 30, 80, 90, 50, 40, 80, 90, 80, 50, 80, 30, 70, 70, 70, 80, 70, 90, 20, 10, 30, 40, 20, 0, 70, 90, 100, 60, 10, 60, 70, 90, 90, 20, 80, 20, 80, 20, 20, 50, 90, 30, 30, 80, 20, 70, 60, 80, 40, 10, 60, 30, 70, 10, 20, 10, 70, 50, 40, 100, 10, 60, 50, 100, 90, 90, 10], [40, 50, 10, 80, 90, 50, 20, 60, 90, 70, 30, 90, 70, 50, 70, 20, 30, 100, 90, 100, 50, 70, 20, 70, 40, 100, 70, 0, 20, 50, 30, 40, 80, 90, 30, 80, 40, 40, 60, 100, 50, 10, 100, 90, 20, 60, 10, 20, 90, 80, 70, 10, 80, 20, 90, 50, 30, 30, 100, 40, 90, 90, 80, 10, 60, 80, 30, 80, 30, 60], [100, 60, 80, 60, 70, 50, 20, 40, 70, 60, 100, 70, 100, 60, 40, 80, 60, 20, 60, 100, 30, 10, 60, 80, 10, 10, 90, 20, 0, 90, 100, 30, 70, 30, 100, 70, 70, 20, 70, 80, 80, 70, 80, 80, 100, 40, 30, 90, 70, 50, 80, 60, 80, 60, 40, 90, 10, 100, 20, 100, 100, 80, 100, 20, 70, 80, 90, 90, 40, 10], [10, 90, 90, 30, 40, 80, 20, 60, 100, 70, 30, 50, 80, 70, 60, 40, 70, 80, 100, 90, 60, 80, 10, 20, 50, 80, 100, 50, 90, 0, 10, 60, 40, 30, 20, 50, 50, 20, 70, 40, 40, 40, 60, 90, 10, 10, 30, 80, 30, 60, 10, 10, 10, 100, 70, 90, 30, 20, 30, 20, 90, 90, 100, 50, 90, 20, 70, 70, 70, 50], [60, 10, 100, 20, 40, 30, 20, 40, 10, 70, 50, 100, 40, 60, 70, 60, 20, 20, 70, 40, 10, 100, 90, 90, 40, 30, 60, 30, 100, 10, 0, 10, 90, 10, 50, 80, 40, 20, 70, 70, 20, 100, 10, 30, 80, 20, 80, 90, 90, 100, 40, 60, 50, 20, 90, 60, 60, 40, 60, 10, 90, 10, 20, 100, 90, 80, 80, 20, 20, 40], [90, 40, 50, 60, 20, 40, 20, 10, 10, 80, 100, 20, 50, 80, 10, 30, 80, 60, 60, 80, 40, 100, 40, 80, 40, 20, 10, 40, 30, 60, 10, 0, 60, 50, 70, 70, 10, 60, 100, 40, 70, 60, 50, 10, 100, 60, 10, 10, 40, 50, 80, 20, 40, 30, 100, 50, 50, 60, 60, 70, 80, 30, 100, 30, 50, 90, 70, 30, 20, 30], [60, 10, 60, 20, 60, 90, 100, 80, 60, 70, 40, 60, 100, 50, 10, 100, 20, 50, 10, 50, 20, 60, 90, 10, 60, 80, 60, 80, 70, 40, 90, 60, 0, 10, 20, 90, 40, 70, 90, 40, 80, 10, 40, 20, 70, 50, 20, 90, 70, 50, 90, 60, 20, 60, 50, 40, 100, 70, 90, 10, 80, 100, 10, 90, 30, 50, 30, 50, 100, 10], [20, 80, 30, 100, 20, 80, 60, 50, 40, 20, 10, 100, 50, 100, 90, 20, 30, 20, 100, 90, 30, 50, 10, 70, 50, 10, 70, 90, 30, 30, 10, 50, 10, 0, 100, 30, 50, 40, 30, 40, 100, 40, 90, 30, 60, 100, 60, 90, 80, 20, 10, 90, 60, 60, 80, 60, 30, 80, 20, 70, 60, 70, 20, 100, 20, 80, 30, 30, 20, 100], [10, 50, 60, 100, 10, 50, 60, 90, 80, 50, 90, 40, 100, 70, 50, 10, 30, 40, 70, 70, 40, 70, 60, 40, 80, 10, 90, 30, 100, 20, 50, 70, 20, 100, 0, 30, 70, 100, 20, 30, 50, 70, 80, 30, 20, 10, 60, 70, 10, 60, 20, 40, 100, 30, 60, 60, 100, 40, 10, 50, 50, 50, 90, 20, 30, 90, 50, 50, 80, 100], [10, 80, 70, 50, 70, 60, 70, 50, 90, 40, 40, 10, 40, 60, 80, 20, 20, 70, 70, 90, 90, 50, 80, 70, 10, 40, 90, 80, 70, 50, 80, 70, 90, 30, 30, 0, 10, 100, 30, 80, 70, 70, 30, 90, 70, 20, 50, 60, 60, 60, 50, 50, 90, 50, 30, 20, 10, 90, 30, 80, 70, 50, 90, 50, 30, 70, 10, 20, 30, 90], [40, 50, 90, 10, 10, 30, 50, 30, 40, 30, 60, 60, 70, 50, 10, 60, 30, 50, 100, 40, 90, 60, 80, 90, 30, 40, 20, 40, 70, 50, 40, 10, 40, 50, 70, 10, 0, 10, 80, 30, 30, 50, 80, 80, 20, 60, 70, 70, 10, 20, 100, 10, 20, 50, 20, 10, 40, 90, 70, 80, 70, 10, 90, 20, 80, 80, 10, 90, 40, 60], [70, 20, 20, 30, 90, 50, 50, 90, 10, 80, 30, 90, 100, 20, 10, 90, 10, 30, 30, 30, 10, 60, 40, 70, 90, 100, 80, 40, 20, 20, 20, 60, 70, 40, 100, 100, 10, 0, 50, 40, 80, 80, 30, 60, 30, 60, 30, 20, 70, 10, 50, 90, 70, 70, 80, 40, 50, 60, 20, 70, 100, 10, 70, 30, 60, 90, 70, 40, 70, 40], [20, 80, 70, 100, 60, 50, 80, 100, 40, 50, 60, 10, 60, 40, 30, 10, 60, 70, 30, 90, 70, 30, 40, 80, 90, 30, 20, 60, 70, 70, 70, 100, 90, 30, 20, 30, 80, 50, 0, 40, 50, 70, 30, 60, 70, 10, 50, 90, 40, 70, 70, 100, 70, 50, 90, 30, 40, 80, 80, 10, 60, 100, 100, 20, 70, 50, 10, 40, 90, 50], [10, 40, 10, 80, 90, 90, 100, 20, 80, 100, 30, 90, 20, 30, 30, 10, 40, 20, 40, 70, 30, 90, 90, 20, 70, 100, 80, 100, 80, 40, 70, 40, 40, 40, 30, 80, 30, 40, 40, 0, 10, 30, 90, 70, 50, 50, 40, 70, 20, 40, 40, 100, 80, 30, 90, 50, 30, 90, 50, 10, 30, 80, 90, 80, 60, 100, 60, 40, 80, 60], [20, 10, 40, 100, 80, 10, 60, 100, 10, 90, 90, 80, 90, 50, 40, 10, 50, 30, 10, 10, 90, 50, 20, 20, 60, 20, 20, 50, 80, 40, 20, 70, 80, 100, 50, 70, 30, 80, 50, 10, 0, 90, 60, 80, 10, 80, 80, 10, 50, 10, 30, 80, 70, 10, 50, 40, 100, 80, 70, 30, 10, 10, 10, 20, 20, 30, 40, 20, 30, 60], [70, 10, 20, 100, 10, 20, 10, 20, 100, 100, 50, 10, 50, 10, 10, 70, 70, 30, 90, 40, 80, 100, 10, 20, 40, 60, 20, 10, 70, 40, 100, 60, 10, 40, 70, 70, 50, 80, 70, 30, 90, 0, 100, 20, 60, 50, 20, 60, 100, 100, 80, 90, 90, 50, 80, 50, 50, 80, 60, 60, 50, 20, 40, 100, 60, 90, 60, 90, 30, 30], [80, 40, 100, 90, 90, 30, 100, 90, 50, 20, 40, 40, 20, 100, 10, 70, 20, 70, 50, 30, 60, 30, 70, 90, 40, 90, 50, 100, 80, 60, 10, 50, 40, 90, 80, 30, 80, 30, 30, 90, 60, 100, 0, 50, 90, 100, 30, 10, 30, 70, 90, 20, 40, 10, 100, 30, 90, 40, 90, 90, 60, 20, 100, 20, 90, 60, 70, 50, 40, 10], [50, 10, 30, 100, 10, 40, 30, 20, 100, 10, 10, 80, 20, 50, 80, 50, 40, 10, 20, 30, 50, 20, 20, 10, 100, 20, 90, 90, 80, 90, 30, 10, 20, 30, 30, 90, 80, 60, 60, 70, 80, 20, 50, 0, 20, 30, 60, 70, 40, 70, 80, 60, 80, 30, 20, 50, 80, 90, 50, 40, 50, 100, 100, 90, 60, 60, 10, 90, 60, 100], [20, 20, 90, 20, 100, 100, 70, 30, 80, 50, 20, 30, 40, 80, 20, 70, 70, 60, 30, 70, 30, 100, 60, 30, 20, 60, 30, 20, 100, 10, 80, 100, 70, 60, 20, 70, 20, 30, 70, 50, 10, 60, 90, 20, 0, 60, 70, 60, 70, 70, 100, 70, 20, 90, 50, 20, 50, 10, 10, 70, 10, 100, 40, 70, 80, 40, 60, 90, 50, 90], [30, 50, 40, 100, 20, 50, 40, 50, 70, 80, 70, 10, 70, 80, 60, 100, 40, 80, 90, 70, 50, 70, 80, 70, 70, 50, 30, 60, 40, 10, 20, 60, 50, 100, 10, 20, 60, 60, 10, 50, 80, 50, 100, 30, 60, 0, 30, 50, 100, 70, 50, 10, 20, 80, 50, 100, 40, 70, 30, 80, 70, 20, 80, 10, 30, 90, 50, 50, 70, 90], [40, 100, 70, 10, 30, 40, 80, 10, 100, 60, 30, 20, 60, 40, 60, 50, 10, 100, 60, 10, 40, 20, 100, 30, 10, 60, 80, 10, 30, 30, 80, 10, 20, 60, 60, 50, 70, 30, 50, 40, 80, 20, 30, 60, 70, 30, 0, 30, 40, 40, 80, 40, 100, 10, 30, 30, 60, 30, 60, 90, 50, 100, 70, 60, 60, 60, 30, 40, 20, 40], [90, 20, 10, 50, 30, 60, 60, 80, 30, 70, 80, 80, 10, 50, 50, 60, 10, 40, 90, 40, 90, 50, 40, 30, 50, 70, 20, 20, 90, 80, 90, 10, 90, 90, 70, 60, 70, 20, 90, 70, 10, 60, 10, 70, 60, 50, 30, 0, 70, 90, 100, 60, 20, 40, 80, 60, 90, 30, 100, 30, 90, 40, 30, 80, 30, 80, 20, 10, 80, 30], [100, 30, 60, 60, 60, 10, 60, 20, 60, 30, 50, 50, 70, 40, 30, 60, 20, 50, 40, 20, 50, 10, 80, 100, 100, 90, 70, 90, 70, 30, 90, 40, 70, 80, 10, 60, 10, 70, 40, 20, 50, 100, 30, 40, 70, 100, 40, 70, 0, 40, 70, 40, 100, 10, 70, 20, 60, 20, 70, 40, 50, 20, 10, 20, 50, 60, 40, 60, 100, 60], [10, 50, 10, 60, 40, 20, 90, 90, 50, 60, 40, 20, 60, 40, 100, 60, 60, 90, 90, 30, 60, 70, 70, 80, 20, 20, 60, 80, 50, 60, 100, 50, 50, 20, 60, 60, 20, 10, 70, 40, 10, 100, 70, 70, 70, 70, 40, 90, 40, 0, 40, 90, 50, 30, 40, 40, 40, 10, 30, 100, 70, 20, 100, 50, 90, 10, 60, 20, 70, 60], [40, 30, 30, 80, 100, 10, 70, 10, 10, 70, 100, 20, 30, 70, 20, 40, 90, 100, 100, 80, 10, 40, 20, 60, 10, 50, 80, 70, 80, 10, 40, 80, 90, 10, 20, 50, 100, 50, 70, 40, 30, 80, 90, 80, 100, 50, 80, 100, 70, 40, 0, 20, 10, 10, 90, 80, 60, 20, 50, 60, 10, 90, 30, 60, 40, 20, 70, 80, 80, 100], [50, 100, 60, 100, 50, 20, 70, 60, 60, 70, 50, 70, 70, 10, 30, 20, 50, 70, 70, 60, 80, 70, 20, 100, 80, 20, 40, 10, 60, 10, 60, 20, 60, 90, 40, 50, 10, 90, 100, 100, 80, 90, 20, 60, 70, 10, 40, 60, 40, 90, 20, 0, 40, 50, 50, 80, 10, 50, 90, 30, 100, 40, 40, 20, 60, 100, 100, 50, 20, 10], [30, 60, 90, 80, 40, 30, 80, 30, 30, 50, 40, 100, 20, 50, 80, 20, 90, 30, 100, 70, 100, 40, 80, 10, 20, 50, 10, 80, 80, 10, 50, 40, 20, 60, 100, 90, 20, 70, 70, 80, 70, 90, 40, 80, 20, 20, 100, 20, 100, 50, 10, 40, 0, 100, 80, 10, 20, 60, 80, 80, 100, 90, 70, 70, 100, 70, 80, 80, 70, 60], [60, 30, 10, 60, 10, 80, 50, 20, 90, 90, 50, 20, 90, 100, 50, 80, 30, 100, 10, 20, 80, 30, 60, 90, 100, 10, 60, 20, 60, 100, 20, 30, 60, 60, 30, 50, 50, 70, 50, 30, 10, 50, 10, 30, 90, 80, 10, 40, 10, 30, 10, 50, 100, 0, 80, 30, 90, 50, 80, 10, 30, 40, 30, 40, 70, 70, 100, 30, 70, 10], [70, 40, 70, 60, 80, 20, 100, 70, 60, 40, 90, 70, 50, 80, 100, 40, 40, 90, 10, 10, 90, 70, 60, 80, 90, 70, 30, 90, 40, 70, 90, 100, 50, 80, 60, 30, 20, 80, 90, 90, 50, 80, 100, 20, 50, 50, 30, 80, 70, 40, 90, 50, 80, 80, 0, 30, 90, 60, 10, 50, 60, 90, 60, 70, 90, 30, 70, 70, 20, 70], [80, 80, 80, 40, 40, 70, 60, 100, 10, 90, 30, 70, 70, 100, 50, 40, 100, 90, 10, 20, 50, 70, 100, 20, 60, 90, 70, 50, 90, 90, 60, 50, 40, 60, 60, 20, 10, 40, 30, 50, 40, 50, 30, 50, 20, 100, 30, 60, 20, 40, 80, 80, 10, 30, 30, 0, 60, 10, 50, 50, 50, 40, 50, 70, 40, 40, 90, 90, 20, 60], [70, 40, 40, 70, 80, 40, 90, 40, 70, 60, 70, 50, 90, 90, 90, 60, 10, 40, 90, 80, 70, 10, 80, 20, 20, 40, 10, 30, 10, 30, 60, 50, 100, 30, 100, 10, 40, 50, 40, 30, 100, 50, 90, 80, 50, 40, 60, 90, 60, 40, 60, 10, 20, 90, 90, 60, 0, 80, 90, 40, 70, 20, 10, 60, 80, 50, 100, 20, 60, 10], [10, 10, 60, 40, 50, 90, 10, 70, 50, 50, 90, 40, 70, 30, 30, 70, 60, 80, 50, 80, 100, 60, 60, 70, 80, 50, 20, 30, 100, 20, 40, 60, 70, 80, 40, 90, 90, 60, 80, 90, 80, 80, 40, 90, 10, 70, 30, 30, 20, 10, 20, 50, 60, 50, 60, 10, 80, 0, 30, 20, 50, 40, 30, 70, 90, 90, 70, 40, 20, 40], [60, 10, 60, 60, 30, 10, 40, 10, 90, 90, 80, 30, 60, 60, 70, 40, 100, 20, 20, 80, 60, 60, 80, 100, 100, 40, 10, 100, 20, 30, 60, 60, 90, 20, 10, 30, 70, 20, 80, 50, 70, 60, 90, 50, 10, 30, 60, 100, 70, 30, 50, 90, 80, 80, 10, 50, 90, 30, 0, 90, 80, 40, 20, 60, 50, 100, 50, 80, 60, 90], [40, 70, 70, 40, 60, 10, 90, 30, 30, 30, 70, 80, 100, 80, 30, 20, 30, 10, 60, 100, 70, 20, 90, 100, 50, 100, 70, 40, 100, 20, 10, 70, 10, 70, 50, 80, 80, 70, 10, 10, 30, 60, 90, 40, 70, 80, 90, 30, 40, 100, 60, 30, 80, 10, 50, 50, 40, 20, 90, 0, 100, 80, 80, 40, 100, 20, 100, 10, 10, 90], [90, 80, 90, 30, 10, 30, 40, 20, 50, 90, 60, 90, 30, 10, 70, 70, 70, 50, 70, 100, 50, 40, 100, 30, 60, 30, 50, 90, 100, 90, 90, 80, 80, 60, 50, 70, 70, 100, 60, 30, 10, 50, 60, 50, 10, 70, 50, 90, 50, 70, 10, 100, 100, 30, 60, 50, 70, 50, 80, 100, 0, 20, 50, 10, 20, 60, 50, 80, 100, 80], [70, 80, 30, 80, 50, 70, 40, 30, 70, 20, 100, 40, 50, 40, 90, 70, 60, 40, 10, 30, 70, 80, 40, 10, 100, 60, 40, 90, 80, 90, 10, 30, 100, 70, 50, 50, 10, 10, 100, 80, 10, 20, 20, 100, 100, 20, 100, 40, 20, 20, 90, 40, 90, 40, 90, 40, 20, 40, 40, 80, 20, 0, 90, 20, 60, 60, 30, 100, 50, 10], [90, 90, 40, 30, 10, 70, 50, 100, 100, 50, 70, 90, 90, 50, 80, 40, 20, 40, 60, 100, 90, 90, 90, 70, 80, 60, 100, 80, 100, 100, 20, 100, 10, 20, 90, 90, 90, 70, 100, 90, 10, 40, 100, 100, 40, 80, 70, 30, 10, 100, 30, 40, 70, 30, 60, 50, 10, 30, 20, 80, 50, 90, 0, 40, 40, 30, 50, 80, 50, 20], [80, 30, 60, 90, 80, 50, 100, 30, 20, 10, 20, 100, 60, 70, 10, 20, 70, 90, 20, 30, 30, 90, 80, 80, 30, 60, 10, 10, 20, 50, 100, 30, 90, 100, 20, 50, 20, 30, 20, 80, 20, 100, 20, 90, 70, 10, 60, 80, 20, 50, 60, 20, 70, 40, 70, 70, 60, 70, 60, 40, 10, 20, 40, 0, 50, 90, 70, 50, 30, 60], [70, 20, 100, 50, 70, 50, 100, 80, 60, 80, 20, 100, 20, 80, 80, 100, 50, 10, 20, 20, 80, 10, 100, 70, 80, 80, 60, 60, 70, 90, 90, 50, 30, 20, 30, 30, 80, 60, 70, 60, 20, 60, 90, 60, 80, 30, 60, 30, 50, 90, 40, 60, 100, 70, 90, 40, 80, 90, 50, 100, 20, 60, 40, 50, 0, 10, 100, 10, 70, 20], [70, 80, 40, 20, 10, 10, 40, 60, 70, 50, 50, 10, 40, 40, 40, 90, 40, 60, 80, 70, 60, 70, 50, 50, 100, 100, 50, 80, 80, 20, 80, 90, 50, 80, 90, 70, 80, 90, 50, 100, 30, 90, 60, 60, 40, 90, 60, 80, 60, 10, 20, 100, 70, 70, 30, 40, 50, 90, 100, 20, 60, 60, 30, 90, 10, 0, 100, 50, 80, 10], [30, 50, 70, 50, 40, 70, 40, 40, 20, 80, 20, 60, 70, 60, 10, 70, 50, 40, 90, 70, 90, 20, 40, 30, 80, 10, 100, 30, 90, 70, 80, 70, 30, 30, 50, 10, 10, 70, 10, 60, 40, 60, 70, 10, 60, 50, 30, 20, 40, 60, 70, 100, 80, 100, 70, 90, 100, 70, 50, 100, 50, 30, 50, 70, 100, 100, 0, 80, 40, 30], [100, 100, 30, 90, 10, 50, 20, 50, 70, 90, 100, 70, 100, 100, 50, 10, 40, 50, 80, 30, 20, 30, 70, 80, 20, 50, 90, 80, 90, 70, 20, 30, 50, 30, 50, 20, 90, 40, 40, 40, 20, 90, 50, 90, 90, 50, 40, 10, 60, 20, 80, 50, 80, 30, 70, 90, 20, 40, 80, 10, 80, 100, 80, 50, 10, 50, 80, 0, 50, 50], [50, 60, 10, 50, 50, 60, 10, 80, 90, 60, 80, 70, 20, 60, 50, 30, 60, 90, 60, 70, 100, 70, 70, 40, 60, 10, 90, 30, 40, 70, 20, 20, 100, 20, 80, 30, 40, 70, 90, 80, 30, 30, 40, 60, 50, 70, 20, 80, 100, 70, 80, 20, 70, 70, 20, 20, 60, 20, 60, 10, 100, 50, 50, 30, 70, 80, 40, 50, 0, 10], [30, 50, 30, 60, 30, 20, 20, 70, 100, 80, 40, 80, 80, 100, 30, 30, 90, 80, 10, 10, 80, 60, 40, 30, 20, 10, 10, 60, 10, 50, 40, 30, 10, 100, 100, 90, 60, 40, 50, 60, 60, 30, 10, 100, 90, 90, 40, 30, 60, 60, 100, 10, 60, 10, 70, 60, 10, 40, 90, 90, 80, 10, 20, 60, 20, 10, 30, 50, 10, 0]]

# discreteTimeWindows = []
# for i in timeWindows:
#     discreteTimeWindows.append(discreteTime(i,10))


In [ ]:
count = 0
nodeCount = []  # assigns a number to node using count and stores its value

for i in range(0,len(discreteTimeWindows)):
    c = []
    for j in range(0, len(discreteTimeWindows[i])):
        c.append(count+1)
        count += 1
    nodeCount.append(c)

print("Node Count ", nodeCount)
print()


edgeFlows = {}
edgeNumber = 0
x = [] 
edges = {}
edgesList = []  # stores edge number going out of ith node
edgesList.append([])  # for s
# solver = pywraplp.Solver.CreateSolver('GLOP')

In [ ]:
e = []
for i in range(0, len(nodeCount)):
            reaching_time = time_from_source[i]
            if(reaching_time <= discreteTimeWindows[i][0]):
                x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
                edges['x[%i]' % edgeNumber] = (0, nodeCount[i][0])
                edgeFlows[edgeNumber] = (0, nodeCount[i][0])
                e.append(edgeNumber)
                edgeNumber += 1
               
    
            else:
                for j in range(0, len(discreteTimeWindows[i])):
                        if(reaching_time <= discreteTimeWindows[i][j]):
                            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
                            edges['x[%i]' % edgeNumber] = (0, nodeCount[i][j])
                            edgeFlows[edgeNumber] = (0, nodeCount[i][j])
                            e.append(edgeNumber)
                            edgeNumber += 1
                            break
edgesList[0] = e

for i in range(0, len(nodeCount)):
    for j in range(0, len(nodeCount[i])-1):
        e = []
#         x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
#         m1.update()
        edges['x[%i]' % edgeNumber] = (nodeCount[i][j], nodeCount[i][j+1])
        edgeFlows[edgeNumber] = (nodeCount[i][j], nodeCount[i][j+1])
        # add edge number in edge list
        e.append(edgeNumber)
        edgesList.append(e)
        edgeNumber += 1
    edgesList.append([])
print("Edges List:", edgesList)

    
for i in range(len(discreteTimeWindows)):
    tu = nodeCount[i][-1]
    start_utime = discreteTimeWindows[i][0]
    end_utime = discreteTimeWindows[i][-1]

    # stores edges reachable from u
    for k in range(0, len(discreteTimeWindows)):
        sv = nodeCount[k][0]

        start_vtime = discreteTimeWindows[k][0]
        end_vtime = discreteTimeWindows[k][-1]


        if(i != k and start_utime + timeMatrix[i][k]+serviceTime > end_vtime):
            print("",end='')
              # not reachable

        elif(i != k and start_vtime > end_utime + timeMatrix[i][k]+ serviceTime):
            # edge bw (u,tu) -> (v,sv)
#             print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)
            e = []
#             x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
            
            edges['x[%i]' % edgeNumber] = (tu, sv)  # u -> v reachable
            edgeFlows[edgeNumber] = (tu, sv)
#             print(tu, "->", sv)
            e.append(edgeNumber)
            edgesList[tu].extend(e)
            edgeNumber += 1


        elif(i != k and start_vtime == end_utime + timeMatrix[i][k]+serviceTime):
#             print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)
            e = []
#             x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
            

            edges['x[%i]' % edgeNumber] = (tu, sv)  # u -> v reachable
#             print(tu, "->", sv)
            edgeFlows[edgeNumber] = (tu, sv)
            e.append(edgeNumber)
            edgesList[tu].extend(e)
            edgeNumber += 1


        elif (i != k and start_vtime < end_utime + timeMatrix[i][k]+serviceTime):
           
            firstReachableNode_index = 0  # first such that node >= sv
            for j in range(0, len(discreteTimeWindows[i])):
                if(timeMatrix[i][k] + discreteTimeWindows[i][j]+serviceTime >= start_vtime):
                    firstReachableNode_index = j
                    break
            if(firstReachableNode_index > 0):
                    firstReachableNode_index=firstReachableNode_index-1
            # now starting from firstReachableNode make edges until feasible
            for l in range(firstReachableNode_index, len(discreteTimeWindows[i])):
                e = []
                reachingTime = discreteTimeWindows[i][l]+timeMatrix[i][k]+serviceTime
                for r in range(0, len(discreteTimeWindows[k])):
                    if(discreteTimeWindows[k][r] >= reachingTime):
                       
                        # edge bw these two
#                         x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
                        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
                        
                        edges['x[%i]' % edgeNumber] =(nodeCount[i][l], nodeCount[k][r])  # u -> v reachable
                        edgeFlows[edgeNumber] = (nodeCount[i][l], nodeCount[k][r])

                        e.append(edgeNumber)
                        edgesList[nodeCount[i][l]].extend(e)
                       
                        edgeNumber += 1
                        break




# print('edge list b4 s t', edgesList)
# print()

# into t (except s)
e = []
for i in range(0, len(nodeCount)):
#     x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
    x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
    edges['x[%i]' % edgeNumber] = (nodeCount[i][-1], -1)
    edgeFlows[edgeNumber] = (nodeCount[i][-1], -1)
    # add edge number in edge list
    edgesList[nodeCount[i][-1]].append(edgeNumber)
    edgeNumber += 1
edgesList.append([])



In [ ]:
noc = 0
# # contraint for outgoing flow for a customer
for i in range(0, len(nodeCount)):  # choose one list from nodeCount
#     contraint = [0] * (edgeNumber)
    coeff = [] # store where coeff = 1
    for j in range(0, len(nodeCount[i])):  # se
        node = nodeCount[i][j]
        for k in range(0, len(edgesList[node])):
            e = edgesList[node][k]
            u = edges['x[%i]' % e][0]
            v = edges['x[%i]' % e][1]
                # print("u", u, "v", v)
            if(v not in nodeCount[i]):
                    # print('hello')
                    # print("u", u, "v", v)
                    coeff.append(x[e])
    
    m1.addConstr(( sum(coeff) == 1  ))
    noc+=1



incomingEdgeList = []
for i in range(0, len(edgesList)):
    incomingEdgeList.append([])  # adding lists in a list
for i in edges:
    v = edges[i][1]  # u -> v

    edge_no = int(i[2:-1])
    incomingEdgeList[v].append(edge_no)

# print("incoming edge list: ", incomingEdgeList)
# print()

#  constraint for incoming for a customer
for i in range(0, len(nodeCount)):
    coeff = []
    contraint = [0] * (edgeNumber)  # indicates number of edges
    for j in range(0, len(nodeCount[i])):
        node = nodeCount[i][j]
        for k in range(0, len(incomingEdgeList[node])):
            e = incomingEdgeList[node][k]
            u = edges['x[%i]' % e][0]
            v = edges['x[%i]' % e][1]
                # print("u", u, "v", v)
            if(u not in nodeCount[i]):
                    # print('hello')
                    # print("u", u, "v", v)
                    coeff.append(x[e])
    m1.addConstr(( sum(coeff) == 1  ))
    noc+=1
    
# constraint for a node incoming = outgoing
for i in range(0, len(nodeCount)):
    for j in range(0, len(nodeCount[i])):
        coeff = []
        contraint = [0] * (edgeNumber)
        node = nodeCount[i][j]
        for k in range(0, len(incomingEdgeList[node])):
            e = incomingEdgeList[node][k]
            contraint[e] = 1
            coeff.append(x[e])
        for k in range(0, len(edgesList[node])):
            e = edgesList[node][k]
            contraint[e] = -1
            coeff.append(-x[e])
        m1.addConstr(( sum(coeff) == 0 ))
        noc+=1  

In [ ]:
objectivefun = []
for i in range(0, len(edgesList[0])):
    objectivefun.append(x[edgesList[0][i]])
obj = sum(objectivefun)
m1.setObjective(obj,GRB.MINIMIZE)
                
# m1.params.Method = 2
# m1.params.Crossover = 0     
# m1.optimize(mycallback)
m1.optimize()
print("Runtime is :",m1.Runtime)
print("objective value : ",m1.objVal)

# for v in m1.getVars():
#     if(v.x == 1):
#         print(v.varName)
# # print('%s: %g' % (v.varName,v.x))

In [ ]:
print(file)
print("Vehicles : ",m1.objVal)
print("Runtime is :",m1.Runtime," seconds")

### 